In [19]:
!pip install numpy
!pip install openai==0.28.0
!pip install opencv-python

  Attempting uninstall: openai
    Found existing installation: openai 1.57.2
    Uninstalling openai-1.57.2:
      Successfully uninstalled openai-1.57.2


In [15]:
import prior
import openai

from integrated_agent import Agent, ACTIONS
from leolani_client import LeolaniChatClient, Action
from ai2thor.controller import Controller

In [16]:
# If you need keys for openai, do so here:
with open('openaikey.txt') as f:
    api_key = f.read().strip()
openai.api_key = api_key

In [17]:
dataset = prior.load_dataset("procthor-10k")
house = dataset["train"][11]
controller = Controller(scene=house, visibilityDistance=3, width=750, height=750)

[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 6361.44it/s]


In [18]:
# Configuration
emissor_path = "./emissor"
HUMAN = "User"
AGENT = "AI2Thor"

# Initialize Leolani client
leolaniClient = LeolaniChatClient(emissor_path=emissor_path, agent=AGENT, human=HUMAN)

# Start the interaction by asking for a human description
actions_available = ", ".join(ACTIONS)
utterance = f"This is what I can do: {actions_available}"
print(AGENT + ">" + utterance)
leolaniClient._add_utterance(AGENT, utterance)
agent = Agent()

# Conversation loop
while not (utterance.lower() in ["stop", "bye", "exit"]):
    agent.process_instruction(utterance)

    # Print and log agent responses
    for ans in agent._answers:
        print(AGENT + ">" + ans)
        leolaniClient._add_utterance(AGENT, ans)

    # Log perceptions
    for obj, objectType, coord, image in agent._perceptions:
        leolaniClient._add_image(obj['name'], objectType, coord, image)
    
    # Log actions (if any were recorded)
    for action in agent._actions:
        leolaniClient._add_action(action)

    utterance = input(HUMAN + "> ")
    leolaniClient._add_utterance(HUMAN, utterance)

# End of conversation
agent.controller.stop()
leolaniClient._save_scenario()
print("Scenario saved and interaction ended.")

AI2Thor>This is what I can do: find, describe, move, turn, head
Error processing the instruction: Unexpected error while parsing instruction: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

AI2Thor>Error processing the instruction: Unexpected error while parsing instruction: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your install

User>  bye


Scenario saved and interaction ended.
